In [1]:
%pylab inline
from __future__ import division
import scipy as sp
from scipy import stats
from scipy.signal import argrelextrema
from scipy.interpolate import interp1d
import operator as op
from decimal import *
# import pickle
from mpl_toolkits.mplot3d import Axes3D
import mpld3
# mpld3.enable_notebook()

Populating the interactive namespace from numpy and matplotlib


In [103]:
%run ../cow.py
%run ../optimize_pkg_2.py

In [3]:
filepath_down = "../data_tables/hs_rs_downlink/n"
filepath_up = "../data_tables/hs_rs_uplink/n"

In [4]:
def xor_analysis_count(N):    
    count = 0
    
    for gc in range(0, N+1):
        for ad in range(0, gc+1):
            for ad_tilde in range(0, ad+1):
                for ad_tilde_s in range(0, ad_tilde+1):
                    for ad_hat_s in range(0, ad-ad_tilde+1):
                        count += 1
    return count

In [5]:
counts = zeros((1, 36))

In [6]:
%%time
for N in range(2, 36):
    counts[0,N] = xor_analysis_count(N)
    print N, counts[0,N]

2 21.0
3 56.0
4 126.0
5 252.0
6 462.0
7 792.0
8 1287.0
9 2002.0
10 3003.0
11 4368.0
12 6188.0
13 8568.0
14 11628.0
15 15504.0
16 20349.0
17 26334.0
18 33649.0
19 42504.0
20 53130.0
21 65780.0
22 80730.0
23 98280.0
24 118755.0
25 142506.0
26 169911.0
27 201376.0
28 237336.0
29 278256.0
30 324632.0
31 376992.0
32 435897.0
33 501942.0
34 575757.0
35 658008.0
CPU times: user 687 ms, sys: 10.8 ms, total: 698 ms
Wall time: 738 ms


In [134]:
z = counts.tolist()[0]
print [int(x) for x in z]

[0, 0, 21, 56, 126, 252, 462, 792, 1287, 2002, 3003, 4368, 6188, 8568, 11628, 15504, 20349, 26334, 33649, 42504, 53130, 65780, 80730, 98280, 118755, 142506, 169911, 201376, 237336, 278256, 324632, 376992, 435897, 501942, 575757, 658008]


In [84]:
def xor_analysis_new(N, tSNR, rSNRdu, rSNR3, p_a1=10**(-9), p_a2=10**(-9), p_a3=10**(-9)):
    if rSNRdu > tSNR: return 0
    h_du = 10**((rSNRdu - tSNR)/10) # linear fade
    h_xor = 10**((rSNR3 - tSNR)/10)
    # Probability fade is bad
    p_f1 = p_f2 = 1 - np.exp(-h_du)
    p_f3 = 1 - np.exp(h_du-h_xor) if h_xor > h_du else 0

    p_link_1 = p_f1 + (1 - p_f1) * p_a1
    p_link_2 = p_f2 + (1 - p_f2) * p_a2
    
    test = []
    
    result = 0
    rv_gc = binom(N, 1 - p_f1)
    for gc in range(0, N+1):
        rv_ad = binom(gc, 1 - p_a1)
        for ad in range(0, gc+1):
            rv_ad_tilde = binom(ad, 1 - p_a2)
            bu = gc - ad
            rv_bu = binom(ad, 1-p_link_2)
            kad = np.arange(1, ad+1)
            s_bu_tilde = (1 - p_f3) + p_f3 * np.dot(rv_bu.pmf(kad), 1-np.power(p_f3, kad)) if p_f3 else 1
            # s_bu_tilde = (1 - p_f3) + p_f3 * sum([rv_bu.pmf(k) * (1 - p_f3**k) for k in range(1, ad+1)]) if p_f3 else 1
            q_bu_tilde = (1 - s_bu_tilde) + s_bu_tilde * p_a3
            for ad_tilde in range(0, ad+1):
                rv_ad_tilde_s = binom(ad_tilde, 1 - p_f3)
                rv_ad_hat_s = binom(ad - ad_tilde, 1 - p_f3)
                # if p_f3 = 0 then ad_tilde_s should = ad_tilde because ad_tilde_i should be empty
                for ad_tilde_s in range(0 if p_f3 else ad_tilde, ad_tilde+1):
                    # ad_tilde already succeeded
                    # if p_f3 = 0 then ad_hat_s should = ad - ad_tilde because ad_hat_i should be empty
                    for ad_hat_s in range(0 if p_f3 else ad-ad_tilde, ad-ad_tilde+1):
                        ad_s = ad_tilde_s + ad_hat_s
                        ad_i = ad - ad_tilde_s - ad_hat_s # ad - ad_s
                        rv_ads = binom(ad_s, 1 - p_link_2)
                        rv_adi = binom(ad_i, 1 - p_link_2)
                        q_ad_hat_s = p_a3
                        q_ad_hat_i = p_link_2**ad_s + (1 - p_link_2)**ad_s * p_a3
                        # This is the problem zone that doesn't support vectorization (?)
                        ks, ki = np.arange(1, ad_s+1), np.arange(0, ad_i+1)
                        f_e = p_link_2**ad_s + (np.dot(rv_ads.pmf(ks), np.power(p_f3, ks)) * np.dot(rv_adi.pmf(ki), np.power(p_f3, ki)) if p_f3 else 0)
                        q_e = f_e + (1 - f_e) * (1 - (1 - p_a3)**2)
                        rv_bu_tilde = binom(bu, 1 - p_a2)
                        bu_tilde = np.arange(0, bu+1)
                        log_pstates = np.log10(rv_gc.pmf(gc)) + np.log10(rv_ad.pmf(ad)) + np.log10(rv_ad_tilde.pmf(ad_tilde)) + np.log10(rv_ad_tilde_s.pmf(ad_tilde_s)) + np.log10(rv_ad_hat_s.pmf(ad_hat_s))
                        bu_tilde_res = np.dot(rv_bu_tilde.pmf(bu_tilde), np.multiply(np.power(1 - q_e, N-ad-bu_tilde), np.power(1 - q_bu_tilde, bu_tilde)))
                        res = 10**log_pstates * (1 - q_ad_hat_s)**ad_hat_s * (1 - q_ad_hat_i)**(ad - ad_tilde - ad_hat_s) * bu_tilde_res
                        result += res
                        # print res, result
                        # print gc, ad, ad_tilde, ad_tilde_s, ad_hat_s, bu_tilde
                        # print "\n"
                        test.append([gc, ad, ad_tilde, ad_tilde_s, ad_hat_s, res, log_pstates])
                        if [gc, ad, ad_tilde, ad_tilde_s, ad_hat_s] == [3, 3, 2, 2, 1]:
                            print [res, np.log10(rv_gc.pmf(gc)), np.log10(rv_ad.pmf(ad)) , np.log10(rv_ad_tilde.pmf(ad_tilde)) , np.log10(rv_ad_tilde_s.pmf(ad_tilde_s)) , np.log10(rv_ad_hat_s.pmf(ad_hat_s))]
#                         test.append([res, np.log10(rv_gc.pmf(gc)), np.log10(rv_ad.pmf(ad)) , np.log10(rv_ad_tilde.pmf(ad_tilde)) , np.log10(rv_ad_tilde_s.pmf(ad_tilde_s)) , np.log10(rv_ad_hat_s.pmf(ad_hat_s))])
    return result, matrix(test)

In [91]:
def xor_analysis_pruned(N, tSNR, rSNRdu, rSNR3, p_a1=10**(-9), p_a2=10**(-9), p_a3=10**(-9)):
    if rSNRdu > tSNR: return 0
    h_du = 10**((rSNRdu - tSNR)/10) # linear fade
    h_xor = 10**((rSNR3 - tSNR)/10)
    # Probability fade is bad
    p_f1 = p_f2 = 1 - np.exp(-h_du)
    p_f3 = 1 - np.exp(h_du-h_xor) if h_xor > h_du else 0

    p_link_1 = p_f1 + (1 - p_f1) * p_a1
    p_link_2 = p_f2 + (1 - p_f2) * p_a2
    
    test = []
#     test2 = []
    pstates_thresh = -14
    
    log_pstates = zeros((5,))
    
    result = 0
    rv_gc = binom(N, 1 - p_f1)
    for gc in range(0, N+1):
        log_pstates[0] = np.log10(rv_gc.pmf(gc))
        rv_ad = binom(gc, 1 - p_a1)
        for ad in range(0, gc+1):
            log_pstates[1] = np.log10(rv_ad.pmf(ad))
            if sum(log_pstates) < pstates_thresh:
                continue
            rv_ad_tilde = binom(ad, 1 - p_a2)
            bu = gc - ad
            rv_bu = binom(ad, 1-p_link_2)
            kad = np.arange(1, ad+1)
            s_bu_tilde = (1 - p_f3) + p_f3 * np.dot(rv_bu.pmf(kad), 1-np.power(p_f3, kad)) if p_f3 else 1
            # s_bu_tilde = (1 - p_f3) + p_f3 * sum([rv_bu.pmf(k) * (1 - p_f3**k) for k in range(1, ad+1)]) if p_f3 else 1
            q_bu_tilde = (1 - s_bu_tilde) + s_bu_tilde * p_a3
            for ad_tilde in range(0, ad+1):
                log_pstates[2] = np.log10(rv_ad_tilde.pmf(ad_tilde))
                if sum(log_pstates) < pstates_thresh:
                    continue
                rv_ad_tilde_s = binom(ad_tilde, 1 - p_f3)
                rv_ad_hat_s = binom(ad - ad_tilde, 1 - p_f3)
                # if p_f3 = 0 then ad_tilde_s should = ad_tilde because ad_tilde_i should be empty
                for ad_tilde_s in range(0 if p_f3 else ad_tilde, ad_tilde+1):
                    log_pstates[3] = np.log10(rv_ad_tilde_s.pmf(ad_tilde_s))
                    if sum(log_pstates) < pstates_thresh:
                        continue
                    # ad_tilde already succeeded
                    # if p_f3 = 0 then ad_hat_s should = ad - ad_tilde because ad_hat_i should be empty
                    for ad_hat_s in range(0 if p_f3 else ad-ad_tilde, ad-ad_tilde+1):
                        log_pstates[4] = np.log10(rv_ad_hat_s.pmf(ad_hat_s))
#                         if [gc, ad, ad_tilde, ad_tilde_s, ad_hat_s] == [3, 3, 2, 2, 1]:
#                             print log_pstates
                        if sum(log_pstates) < pstates_thresh:
                            continue
                        ad_s = ad_tilde_s + ad_hat_s
                        ad_i = ad - ad_tilde_s - ad_hat_s # ad - ad_s
                        rv_ads = binom(ad_s, 1 - p_link_2)
                        rv_adi = binom(ad_i, 1 - p_link_2)
                        q_ad_hat_s = p_a3
                        q_ad_hat_i = p_link_2**ad_s + (1 - p_link_2)**ad_s * p_a3
                        # This is the problem zone that doesn't support vectorization (?)
                        ks, ki = np.arange(1, ad_s+1), np.arange(0, ad_i+1)
                        f_e = p_link_2**ad_s + (np.dot(rv_ads.pmf(ks), np.power(p_f3, ks)) * np.dot(rv_adi.pmf(ki), np.power(p_f3, ki)) if p_f3 else 0)
                        q_e = f_e + (1 - f_e) * (1 - (1 - p_a3)**2)
                        rv_bu_tilde = binom(bu, 1 - p_a2)
                        bu_tilde = np.arange(0, bu+1)
#                         log_pstates = np.log10(rv_gc.pmf(gc)) + np.log10(rv_ad.pmf(ad)) + np.log10(rv_ad_tilde.pmf(ad_tilde)) + np.log10(rv_ad_tilde_s.pmf(ad_tilde_s)) + np.log10(rv_ad_hat_s.pmf(ad_hat_s))
                        bu_tilde_res = np.dot(rv_bu_tilde.pmf(bu_tilde), np.multiply(np.power(1 - q_e, N-ad-bu_tilde), np.power(1 - q_bu_tilde, bu_tilde)))
                        res = 10**sum(log_pstates) * (1 - q_ad_hat_s)**ad_hat_s * (1 - q_ad_hat_i)**(ad - ad_tilde - ad_hat_s) * bu_tilde_res
                        result += res
                        # print res, result
                        # print gc, ad, ad_tilde, ad_tilde_s, ad_hat_s, bu_tilde
                        # print "\n"
                        test.append([gc, ad, ad_tilde, ad_tilde_s, ad_hat_s, res, sum(log_pstates)])
#                         test.append([res, log_pstates, (1 - q_ad_hat_s)**ad_hat_s, (1 - q_ad_hat_i)**(ad - ad_tilde - ad_hat_s), bu_tilde_res])
    return result, matrix(test)

In [85]:
%%time
downbit = upbit = xorbit = 4*10**4/3
N = 4

downTable = load_table(filepath_down+str(N)+extension)
downfunc = interp1d(downTable[0], downTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))
upTable = load_table(filepath_up+str(N)+extension)
upfunc = interp1d(upTable[0], upTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

xor_table = load_table(filepath_up+str(N)+extension)
xor_func = interp1d(xor_table[0], xor_table[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

tSNR = 26.9
rSNRdu = 0.7
rSNR3 = 0.8
xor_opt, histdata = xor_analysis_new(N, tSNR, rSNRdu, rSNR3, downfunc(rSNRdu), upfunc(rSNRdu), xor_func(rSNR3))

[2.923257648330359e-12, -2.0215862041918036, 0.0, -9.5124738948521923, -4.8533274045399815e-05, -2.426663702270604e-05]
CPU times: user 475 ms, sys: 6.05 ms, total: 481 ms
Wall time: 495 ms


/Users/leahdickstein/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:46: RuntimeWarning: divide by zero encountered in log10


In [49]:
print 1 - xor_opt

1.01895014648e-09


In [50]:
histdata=matrix(histdata)
print np.amin(histdata[:,-1])
# print np.amax(histdata[:,0])
print histdata[:,0:3]

-17.011147402
[[  0.00000000e+000  -1.04820832e+001   0.00000000e+000]
 [  0.00000000e+000  -7.26054420e+000              -inf]
 [ -0.00000000e+000  -7.26054420e+000   0.00000000e+000]
 [  5.58005125e-018  -7.26054420e+000   0.00000000e+000]
 [  0.00000000e+000  -7.26054420e+000   0.00000000e+000]
 [  5.44795282e-008  -7.26054420e+000   0.00000000e+000]
 [  0.00000000e+000  -4.46497394e+000              -inf]
 [ -0.00000000e+000  -4.46497394e+000              -inf]
 [  0.00000000e+000  -4.46497394e+000              -inf]
 [  0.00000000e+000  -4.46497394e+000              -inf]
 [  0.00000000e+000  -4.46497394e+000              -inf]
 [  0.00000000e+000  -4.46497394e+000   0.00000000e+000]
 [  3.98959784e-029  -4.46497394e+000   0.00000000e+000]
 [  3.59568804e-025  -4.46497394e+000   0.00000000e+000]
 [ -0.00000000e+000  -4.46497394e+000   0.00000000e+000]
 [  3.90450577e-019  -4.46497394e+000   0.00000000e+000]
 [  3.89515074e-019  -4.46497394e+000   0.00000000e+000]
 [  7.02113221e-0

In [70]:
print np.where(histdata[:,-2] > 10**(-14))[0]
print len(np.where(histdata[:,-2] > 10**(-14))[0])
print histdata[np.where(histdata[:,-2] > 10**(-14))[0],:]
# print np.amin(histdata[np.where(histdata[:,-2] > 10**(-14))[0],-1])

[  5  19  20  51  53  54  55 118 119 120 122 123 124 125]
14
[[  1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
    0.00000000e+00   5.44795282e-08  -7.26056846e+00]
 [  2.00000000e+00   2.00000000e+00   2.00000000e+00   1.00000000e+00
    0.00000000e+00   3.81207309e-09  -8.41675506e+00]
 [  2.00000000e+00   2.00000000e+00   2.00000000e+00   2.00000000e+00
    0.00000000e+00   3.42745929e-05  -4.46502247e+00]
 [  3.00000000e+00   3.00000000e+00   2.00000000e+00   2.00000000e+00
    1.00000000e+00   2.92325765e-12  -1.15341329e+01]
 [  3.00000000e+00   3.00000000e+00   3.00000000e+00   1.00000000e+00
    0.00000000e+00   8.88986739e-11  -1.00500629e+01]
 [  3.00000000e+00   3.00000000e+00   3.00000000e+00   2.00000000e+00
    0.00000000e+00   1.59476675e-06  -5.79730033e+00]
 [  3.00000000e+00   3.00000000e+00   3.00000000e+00   3.00000000e+00
    0.00000000e+00   9.51351461e-03  -2.02165900e+00]
 [  4.00000000e+00   4.00000000e+00   3.00000000e+00   2.00000000e+00
 

In [15]:
print histdata[5,-1]

-7.26056846271


In [92]:
%%time
downbit = upbit = xorbit = 4*10**4/3
N = 4

downTable = load_table(filepath_down+str(N)+extension)
downfunc = interp1d(downTable[0], downTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))
upTable = load_table(filepath_up+str(N)+extension)
upfunc = interp1d(upTable[0], upTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

xor_table = load_table(filepath_up+str(N)+extension)
xor_func = interp1d(xor_table[0], xor_table[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

tSNR = 27
rSNRdu = 0.7
rSNR3 = 0.8
xor_opt, histdata = xor_analysis_pruned(N, tSNR, rSNRdu, rSNR3, downfunc(rSNRdu), upfunc(rSNRdu), xor_func(rSNR3))

CPU times: user 110 ms, sys: 1.86 ms, total: 112 ms
Wall time: 112 ms


/Users/leahdickstein/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in log10


In [94]:
histdata = matrix(histdata)
print 1 - xor_opt
print shape(histdata)
# print np.amin(histdata[:,1])
print histdata

9.29523902293e-10
(17, 7)
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00  -1.05220358e+01]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00  -1.15532715e+01]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
    0.00000000e+00   5.08587827e-08  -7.29050864e+00]
 [  2.00000000e+00   2.00000000e+00   2.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00  -1.30104760e+01]
 [  2.00000000e+00   2.00000000e+00   2.00000000e+00   1.00000000e+00
    0.00000000e+00   3.55861180e-09  -8.44668310e+00]
 [  2.00000000e+00   2.00000000e+00   2.00000000e+00   2.00000000e+00
    0.00000000e+00   3.27374447e-05  -4.48495024e+00]
 [  3.00000000e+00   3.00000000e+00   2.00000000e+00   2.00000000e+00
    1.00000000e+00   2.85727307e-12  -1.15440482e+01]
 [  3.00000000e+00   3.00000000e+00   3.00000000e+00   1.00000000e+00
    0.00000000e+00   8.29856685e-11 

In [93]:
# Pruned
# print histdata[:,0:5]
print histdata[np.where(histdata[:,-2] > 10**(-14))[0],0:5]

[[ 1.  1.  1.  1.  0.]
 [ 2.  2.  2.  1.  0.]
 [ 2.  2.  2.  2.  0.]
 [ 3.  3.  2.  2.  1.]
 [ 3.  3.  3.  1.  0.]
 [ 3.  3.  3.  2.  0.]
 [ 3.  3.  3.  3.  0.]
 [ 4.  4.  3.  2.  1.]
 [ 4.  4.  3.  3.  0.]
 [ 4.  4.  3.  3.  1.]
 [ 4.  4.  4.  1.  0.]
 [ 4.  4.  4.  2.  0.]
 [ 4.  4.  4.  3.  0.]
 [ 4.  4.  4.  4.  0.]]


In [68]:
# Control 
print histdata[np.where(histdata[:,-2] > 10**(-14))[0],0:5]

# matches!!

[[ 1.  1.  1.  1.  0.]
 [ 2.  2.  2.  1.  0.]
 [ 2.  2.  2.  2.  0.]
 [ 3.  3.  2.  2.  1.]
 [ 3.  3.  3.  1.  0.]
 [ 3.  3.  3.  2.  0.]
 [ 3.  3.  3.  3.  0.]
 [ 4.  4.  3.  2.  1.]
 [ 4.  4.  3.  3.  0.]
 [ 4.  4.  3.  3.  1.]
 [ 4.  4.  4.  1.  0.]
 [ 4.  4.  4.  2.  0.]
 [ 4.  4.  4.  3.  0.]
 [ 4.  4.  4.  4.  0.]]


In [114]:
print 1 - xor_opt

9.44130551517e-10


In [105]:
%%time
%run ../optimize_pkg_2.py
downbit = upbit = xorbit = 4*10**4/3
N = 5

downTable = load_table(filepath_down+str(N)+extension)
downfunc = interp1d(downTable[0], downTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))
upTable = load_table(filepath_up+str(N)+extension)
upfunc = interp1d(upTable[0], upTable[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

xor_table = load_table(filepath_up+str(N)+extension)
xor_func = interp1d(xor_table[0], xor_table[1], kind='linear', bounds_error=False, fill_value=(1.0, 0.0))

tSNR = 23
rSNRdu = 0
rSNR3 = 1
xor_opt = xor_analysis_pruned(N, tSNR, rSNRdu, rSNR3, downfunc(rSNRdu), upfunc(rSNRdu), xor_func(rSNR3))

CPU times: user 304 ms, sys: 2.39 ms, total: 306 ms
Wall time: 309 ms


In [121]:
%%time
%run ../optimize_pkg_2.py
N = 5
tSNR_start = 22.5
rSNR_start = 0
rSNR3_start = 0.5

protocol_target = 10**(-9)
protocolbit = 4*10**4

precision = -1
d_rSNR = 0.1

filepath_down = "../data_tables/hs_rs_downlink/n"
filepath_up = "../data_tables/hs_rs_uplink/n"

tx_SNR_range = np.arange(tSNR_start, 50, 0.1)
result = optimize_3(N, rSNR_start, rSNR3_start, tx_SNR_range, filepath_down, filepath_up, protocolbit, d_rSNR, precision)
print result

<string>:181: RuntimeWarning: divide by zero encountered in log10
<string>:192: RuntimeWarning: divide by zero encountered in log10


(22.600000000000001, 0.0, 0.89999999999999991, array(3.569272834933721e-14), array(0.0005035279292351958), array(1.0873919099203486e-09))
CPU times: user 20min 1s, sys: 5.29 s, total: 20min 6s
Wall time: 30min 25s


In [115]:
%%time
%run ../optimize_pkg_2.py
optimize_3_data_2 = zeros((5, 6))
# tx_SNR_range = arange(30, 70, 1)
tx_SNR_range = np.arange(tSNR_start, 50, 1)
rx_SNR_start = 1
# tx_SNR_range = arange(32, 60, 0.01)
for N in [5]:
    optimize_3_data_2[N-1] = optimize_3(N, rSNR_start, rSNR3_start, tx_SNR_range, filepath_down, filepath_up, protocolbit, d_rSNR, precision)
    print N, optimize_3_data_2[N-1]

Hi
5 [ nan  nan  nan  nan  nan  nan]
CPU times: user 38.4 ms, sys: 5.76 ms, total: 44.1 ms
Wall time: 54.3 ms


In [128]:
%%time
%run ../optimize_pkg_2.py
# new version with early break
N = 5
tSNR_start = 22.5
rSNRdu_start = -0.1
rSNR3_start = 0.8

protocol_target = 10**(-9)
protocolbit = 4*10**4

precision = -2
d_rSNR = 0.1

filepath_down = "../data_tables/hs_rs_downlink/n"
filepath_up = "../data_tables/hs_rs_uplink/n"

tx_SNR_range = np.arange(tSNR_start, 50, 0.01)
result = optimize_3(N, rSNRdu_start, rSNR3_start, tx_SNR_range, filepath_down, filepath_up, protocolbit, d_rSNR, precision)
print result

(22.520000000000003, 0.10000000000000001, 0.90000000000000002, array(1.916959323832033e-16), array(0.00016973866892785626), array(1.0873919099203466e-09))
CPU times: user 1min 59s, sys: 1.14 s, total: 2min
Wall time: 2min 7s


<string>:181: RuntimeWarning: divide by zero encountered in log10


In [127]:
%%time
%run ../optimize_pkg_2.py
# new version with early break
N = 5
tSNR_start = 22.5
rSNRdu_start = -0.1
rSNR3_start = 0.8

protocol_target = 10**(-9)
protocolbit = 4*10**4

precision = -2
d_rSNR = 0.01

filepath_down = "../data_tables/hs_rs_downlink/n"
filepath_up = "../data_tables/hs_rs_uplink/n"

tx_SNR_range = np.arange(tSNR_start, 50, 0.01)
result = optimize_3(N, rSNRdu_start, rSNR3_start, tx_SNR_range, filepath_down, filepath_up, protocolbit, d_rSNR, precision)
print result

(22.510000000000002, 0.1099999999999999, 0.90000000000000013, array(1.1087336607542678e-16), array(0.000151523847246746), array(1.0873919099203445e-09))
CPU times: user 15min 13s, sys: 4.33 s, total: 15min 17s
Wall time: 15min 33s


<string>:181: RuntimeWarning: divide by zero encountered in log10
